In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
!pwd
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')
# %load_ext autoreload
# %autoreload 2


from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv

C:\Users\Egor\Dropbox\GitHub\Mosaic\msq-domain\notebooks


In [2]:
import pickle

try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
    filename = '../resources/fx/EUR_JPY_GBP_till_SN.csv'
    quote_dict = qc_csv_helper.get_fx_quotes(filename)
    with open('data.pickle', 'wb') as f:\
        pickle.dump(quote_dict, f)

In [4]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_fx_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_fx_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)

my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  venue = None, notionals = (100, -111.97999999999999), static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x000001E50621DF28>, settle_date = 2017-06-26, isDeliverable = True, tenor = SPOT, holidayCities = None
<class 'mosaicsmartdata.core.trade.FXForwardTrade'> :  factor_risk = None, adj_traded_px = None, timestamp = 2017-06-26 00:02:26.995000, package = None, trade_id = 12345, leg_no = None, source_trade_id = None, trade_settle_date = None, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  venue = None, notionals = (100, -111.97999999999999), static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x000001E50621DF28>, settle_date = 2017-06-26, isDeliverable = True, tenor = SPOT, holidayCities = None, paper_trade = Fa

In [5]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [6]:

logging.getLogger().setLevel('INFO')
logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes, trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([0, 10, 100]))\
            > print
        
run(stream)



INFO:root:starting command line test...
INFO:aiostreams.runner:Creating a runnable graph with source <aiostreams.operators.flatten.Flatten object at 0x000001E5073350F0> and sink <built-in function print>
INFO:aiostreams.runner:Attempting to run the graphs ["{'ignore_heartbeats': True, 'source': <aiostreams.operators.flatten.Flatten object at 0x000001E5073350F0>, 'persistence_policy': None, 'persister_factory': <class 'aiostreams.persist.KafkaPersister'>, 'log_level': 10, 'sink': <built-in function print>, 'id': 'pipe_994160', 'message_count': 0, 'logger': <logging.Logger object at 0x000001E507333E10>, 'persister': None}; "]
INFO:aiostreams.runner:Graph pipe_994160 creating a coroutine for the event loop


 dt:0 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:26.995000 package:None PV:5.0000000000105516e-05 price:5.0000000000105516e-05 timestamp:2017-06-26 00:02:27.165000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:1.1198 ccy:('EUR', 'USD')
 dt:10 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:36.995000 package:None PV:5.0000000000105516e-05 price:5.0000000000105516e-05 timestamp:2017-06-26 00:02:37.244000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:1.1198 ccy:('EUR', 'USD')
 dt:100 final_price:1.12 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:04:06.995000 package:None PV:0.00020000000

INFO:aiostreams.runner:Finished running graphs ["{'ignore_heartbeats': True, 'source': <aiostreams.operators.flatten.Flatten object at 0x000001E5073350F0>, 'persistence_policy': None, 'persister_factory': <class 'aiostreams.persist.KafkaPersister'>, 'last_msg': <mosaicsmartdata.core.markout_msg.MarkoutMessage2 object at 0x000001E507335978>, 'log_level': 10, 'sink': <built-in function print>, 'id': 'pipe_994160', 'message_count': 3, 'logger': <logging.Logger object at 0x000001E507333E10>, 'persister': None} producing 3 messages; "]
